In [1]:
import sgml, sgutil, sgpp, dproc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import root_mean_squared_error

from proc_v2 import p
from ml_v2 import X_cat, X_num, X_all, target, kf, skf, ss, config, scheduler
from ml_v2 import xgb_adapter, lgb_adapter, cb_adapter, lr_adapter, nn_adapter

sc = sgutil.SGCache('img', 'result', 'model')
df_train = p.fit_transform(['data/train.csv']).assign(
    Calories_Log = lambda x: np.log(x['Calories'] + 1)
)
df_test = p.transform(['data/test.csv'])

2025-05-26 00:37:29.074539: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-26 00:37:29.083348: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748187449.092866  259059 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748187449.095950  259059 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1748187449.103642  259059 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [2]:
df_train['Duration'].nunique()

30

In [3]:
df_train['Heart_Rate'].nunique()

63

In [4]:
df_train['Body_Temp'].nunique()

75

# Target Encoder의 효과를 봅니다.

In [5]:
df_train.groupby(['Duration', 'Heart_Rate', 'Body_Temp'])[target].agg(
    ['mean', 'std', 'size']
).query('size >= 50')['std'].mean()

np.float32(0.12801187)

In [6]:
df_train.groupby(['Duration', 'Heart_Rate'])[target].agg(
    ['mean', 'std', 'size']
).query('size >= 50')['std'].mean()

np.float32(0.14161234)

In [7]:
df_train.groupby(['Duration', 'Body_Temp'])[target].agg(
    ['mean', 'std', 'size']
).query('size >= 50')['std'].mean()

np.float32(0.1881298)

In [8]:
df_train.groupby(['Duration'])[target].agg(
    ['mean', 'std', 'size']
).query('size >= 50')['std'].mean()

np.float32(0.17675276)

In [9]:
df_train.groupby(['Sex', 'Duration', 'Heart_Rate', 'Body_Temp'])[target].agg(
    ['mean', 'std', 'size']
).query('size >= 50')['std'].mean()

np.float32(0.10964416)

In [10]:
df_train.groupby(['Height', 'Weight'])[target].agg(
    ['mean', 'std', 'size']
).query('size >= 50')['std'].mean()

np.float32(0.9575898)

In [11]:
df_train.groupby(['Height'])[target].agg(
    ['mean', 'std', 'size']
).query('size >= 50')['std'].mean()

np.float32(0.9839047)

In [12]:
df_train.groupby(['Weight'])[target].agg(
    ['mean', 'std', 'size']
).query('size >= 50')['std'].mean()

np.float32(1.0028571)

In [13]:
df_train.groupby(['Sex', 'Height', 'Weight'])[target].agg(
    ['mean', 'std', 'size']
).query('size >= 50')['std'].mean()

np.float32(0.9559728)

In [6]:
cc = sgpp.CatCombiner2(
    [['Sex', 'Duration', 'Heart_Rate', 'Body_Temp'], ['Sex', 'Height', 'Weight']]
)
df_train = dproc.join_and_assign(
    df_train, cc.fit_transform(df_train)
)

- Age를 제외하고 log 속성을 추가하면 성능 향상이 있습니다.

In [8]:
sgml.cv(df_train, kf, {
    'X_std': ['Duration_log', 'Heart_Rate_sqrt_d', 'Height_log', 'Weight_log', 'Heart_Rate_log', 'Body_Temp_log'] + X_num, 
    'X_tgt': df_train.columns[-2:].tolist(), 'tgt': {'smooth': 0.1, 'random_state': 123}}, config, lr_adapter)

Fold:   0%|          | 0/4 [00:00<?, ?it/s]

{'valid_scores': [0.0989459678530693,
  0.0982351079583168,
  0.09807074815034866,
  0.09894682466983795],
 'valid_prd': id
 0         4.990277
 1         3.603220
 2         3.370885
 3         4.922816
 4         4.987004
             ...   
 749995    5.351160
 749996    4.581959
 749997    5.523477
 749998    4.622436
 749999    4.605652
 Length: 750000, dtype: float32,
 'model_result': [],
 'train_scores': [0.09841784834861755,
  0.09865152835845947,
  0.098708875477314,
  0.09841430932283401],
 'hparams': {'X_std': ['Duration_log',
   'Heart_Rate_sqrt_d',
   'Height_log',
   'Weight_log',
   'Heart_Rate_log',
   'Body_Temp_log',
   'Age',
   'Height',
   'Weight',
   'Duration',
   'Heart_Rate',
   'Body_Temp'],
  'tgt': {'smooth': 0.1, 'random_state': 123}}}

# NN4

In [5]:
X_log = ['Duration_log', 'Heart_Rate_sqrt_d', 'Height_log', 'Weight_log', 'Heart_Rate_log', 'Body_Temp_log']
nn_params = {
    'config':  [
        {'unit': 128, 'activation': 'swish', 'batch_norm': False},
        {'unit': 256, 'activation': 'swish', 'batch_norm': False},
        {'unit': 256, 'activation': 'swish', 'batch_norm': False},
        {'unit': 128, 'activation': 'swish', 'batch_norm': False},
    ]
}
hparams = {
    'model_params': {
        'model_params': nn_params,
        'epochs': 30,
        'optimizer': ('Adam', {'learning_rate': 0.0001}),
        'batch_size': 128, 'shuffle_size': 204800,
        'early_stopping': None, 'reduce_lr_on_plateau': None, 'lr_scheduler': {'schedule': scheduler}
    }, 'X_std': ['Age', 'Height', 'Weight', 'Duration', 'Heart_Rate', 'Heart_Rate_sqrt_d', 'Body_Temp'] + X_log, 'X_num': ['Sex']
}

result = sc.cv_result('nn4', df_train, skf, hparams, config, nn_adapter, rerun = 0)

Fold:   0%|          | 0/5 [00:00<?, ?it/s]

Epoch:   0%|          | 0/30 [00:00<?, ?it/s]

Step:   0%|          | 0/4688 [00:00<?, ?it/s]

Epoch:   0%|          | 0/30 [00:00<?, ?it/s]

Step:   0%|          | 0/4688 [00:00<?, ?it/s]

...

Step:   0%|          | 0/4688 [00:00<?, ?it/s]

Epoch:   0%|          | 0/30 [00:00<?, ?it/s]

Step:   0%|          | 0/4688 [00:00<?, ?it/s]

Epoch:   0%|          | 0/30 [00:00<?, ?it/s]

Step:   0%|          | 0/4688 [00:00<?, ?it/s]

In [6]:
np.mean(result['valid_scores']), result['valid_scores']

(0.05956732928752899,
 [0.05952732264995575,
  0.059361450374126434,
  0.060435764491558075,
  0.059219636023044586,
  0.059292472898960114])

In [4]:
df_train['Age'] // 10

id
0         3
1         6
2         5
3         2
4         3
         ..
749995    2
749996    6
749997    6
749998    4
749999    3
Name: Age, Length: 750000, dtype: int8

In [10]:
df_train['Dur_target_std'] = df_train.groupby('Duration')[target].transform(lambda x: (x - x.mean()) / x.std())

In [43]:
df_train[['Height', 'Weight', 'Duration', 'Age', 'Body_Temp', 'Heart_Rate', 'Sex', 'Dur_target_std']].corr()['Dur_target_std']

Height           -5.778868e-03
Weight            3.567946e-02
Duration         -3.857965e-07
Age               5.584026e-01
Body_Temp         2.224439e-02
Heart_Rate        3.200603e-01
Sex              -4.987826e-02
Dur_target_std    1.000000e+00
Name: Dur_target_std, dtype: float64

In [19]:
from scipy.stats import pearsonr
pearsonr(df_train['Duration'], df_train[target])

PearsonRResult(statistic=0.94012845, pvalue=0.0)

In [20]:
pearsonr(df_train['Dur_target_std'], df_train['Age'])

PearsonRResult(statistic=0.5584028, pvalue=0.0)

In [2]:
df_train['Age_2'] = df_train['Age'].clip(0, 65)

In [115]:
df_train['DA_target_std'] = df_train.groupby(['Duration', 'Age_2'])[target].transform(lambda x: (x - x.mean()) / x.std())

In [116]:
df_train[['Height', 'Weight','Body_Temp', 'Heart_Rate', 'Sex', 'DA_target_std']].corr()['DA_target_std']

Height           0.005997
Weight           0.015812
Body_Temp        0.017662
Heart_Rate       0.390577
Sex             -0.038893
DA_target_std    1.000000
Name: DA_target_std, dtype: float64

In [117]:
df_train[['Duration', 'Age_2', 'Sex']].value_counts()

Duration  Age_2  Sex  
29.0      65     True     1882
28.0      65     True     1763
26.0      65     True     1660
27.0      65     True     1517
10.0      65     False    1402
                          ... 
1.0       55     False      33
          58     False      32
22.0      60     False      30
1.0       49     False      29
          53     False      24
Name: count, Length: 2760, dtype: int64

In [3]:
df_train['DAS_target_std'] = df_train.groupby(['Duration', 'Age_2', 'Sex'])[target].transform(lambda x: (x - x.mean()) / x.std())

In [119]:
df_train[['Height', 'Weight','Body_Temp', 'Heart_Rate', 'DAS_target_std']].corr()['DAS_target_std']

Height            0.017093
Weight            0.028878
Body_Temp         0.014610
Heart_Rate        0.427700
DAS_target_std    1.000000
Name: DAS_target_std, dtype: float64

In [5]:
df_train['DAS'] = df_train[['Duration', 'Age_2', 'Sex']].astype(str).sum(axis = 1).astype('category')

In [121]:
from sklearn.preprocessing import TargetEncoder

tgt = TargetEncoder()
root_mean_squared_error(
    df_train[target], tgt.fit_transform(df_train[['DAS']], df_train[target])[:, 0]
)

0.14046500059439904

In [122]:
X_num

['Age', 'Height', 'Weight', 'Duration', 'Heart_Rate', 'Body_Temp']

In [124]:
X_log = ['Duration_log', 'Height_log', 'Weight_log', 'Heart_Rate_log', 'Body_Temp_log']
sgml.cv(df_train, skf, {
    'X_std': ['Age', 'Height', 'Weight', 'Duration', 'Heart_Rate', 'Heart_Rate_sqrt_d', 'Body_Temp'] + X_log, 
    'X_tgt': ['DAS'], 'tgt': {'random_state': 123}}, config, lr_adapter)

Fold:   0%|          | 0/5 [00:00<?, ?it/s]

{'valid_scores': [0.07191105807652111,
  0.07180225181490403,
  0.07281966892552419,
  0.07168646259620916,
  0.07111156994542099],
 'valid_prd': id
 0         4.995144
 1         3.569119
 2         3.356816
 3         4.912358
 4         4.978397
             ...   
 749995    5.340399
 749996    4.523926
 749997    5.495052
 749998    4.684679
 749999    4.614335
 Length: 750000, dtype: float64,
 'model_result': [],
 'train_scores': [0.0715751222425497,
  0.07158818967978951,
  0.07133095948730765,
  0.07157798322156914,
  0.07177942867486596],
 'hparams': {'X_std': ['Age',
   'Height',
   'Weight',
   'Duration',
   'Heart_Rate',
   'Heart_Rate_sqrt_d',
   'Body_Temp',
   'Duration_log',
   'Height_log',
   'Weight_log',
   'Heart_Rate_log',
   'Body_Temp_log'],
  'X_tgt': ['DAS'],
  'tgt': {'random_state': 123}}}

In [49]:
X_log = ['Duration_log', 'Height_log', 'Weight_log', 'Heart_Rate_log', 'Body_Temp_log']
nn_params = {
    'config':  [
        {'unit': 128, 'activation': 'swish', 'batch_norm': False},
        {'unit': 256, 'activation': 'swish', 'batch_norm': False},
        {'unit': 256, 'activation': 'swish', 'batch_norm': False},
        {'unit': 128, 'activation': 'swish', 'batch_norm': False},
    ]
}
hparams = {
    'model_params': {
        'model_params': nn_params,
        'epochs': 30,
        'optimizer': ('Adam', {'learning_rate': 0.0001}),
        'batch_size': 128, 'shuffle_size': 204800,
        'early_stopping': None, 'reduce_lr_on_plateau': None, 'lr_scheduler': {'schedule': scheduler}
    }, 'X_std': ['Age', 'Height', 'Weight', 'Duration', 'Duration_log', 'Heart_Rate', 'Heart_Rate_sqrt_d', 'Body_Temp'], 'X_num': ['Sex'],
    'X_tgt': ['DAS'], 'tgt': {'random_state': 123}
}

result = sc.cv_result('nn5', df_train, skf, hparams, config, nn_adapter, rerun = 1)

Fold:   0%|          | 0/5 [00:00<?, ?it/s]

Epoch:   0%|          | 0/30 [00:00<?, ?it/s]

Step:   0%|          | 0/4688 [00:00<?, ?it/s]

Epoch:   0%|          | 0/30 [00:00<?, ?it/s]

Step:   0%|          | 0/4688 [00:00<?, ?it/s]

...

Step:   0%|          | 0/4688 [00:00<?, ?it/s]

Epoch:   0%|          | 0/30 [00:00<?, ?it/s]

Step:   0%|          | 0/4688 [00:00<?, ?it/s]

Epoch:   0%|          | 0/30 [00:00<?, ?it/s]

Step:   0%|          | 0/4688 [00:00<?, ?it/s]

In [50]:
np.mean(result['valid_scores'])

0.059637806564569476

## CB6

- DAS 를 CatBoost에 사용해봅니다.

In [16]:
hparams = {
    'model_params': {'max_depth': 7, 'n_estimators': 2000, 'learning_rate': 0.07, 'colsample_bylevel': 0.8}, 
    'X_num': ['Sex', 'Age', 'Height', 'Weight', 'Duration_log', 'Heart_Rate', 'Body_Temp'], 
    'X_tgt': ['DAS'], 'tgt': {'random_state': 123},
    'validation_fraction': 'oof'
}

result = sc.cv_result('cb6', df_train, skf, hparams, config, cb_adapter, use_gpu = 0, rerun = 1)
np.mean(result['valid_scores']), result['valid_scores']

Fold:   0%|          | 0/5 [00:00<?, ?it/s]

(0.05957448612322343,
 [0.060064969146597355,
  0.05955261904152514,
  0.05978967750250761,
  0.05926171406685314,
  0.05920345085863389])

# LGB4

In [18]:
hparams = {
    'model_params': {'n_estimators': 2500, 'colsample_bytree': 0.9, 'num_leaves': 15, 'learning_rate': 0.07},
    'X_num': ['Sex', 'Age', 'Height', 'Weight', 'Duration_log', 'Heart_Rate', 'Body_Temp'], 
    'X_tgt': ['DAS'], 'tgt': {'random_state': 123},
    'validation_fraction': 'oof', 
}
result = sc.cv_result('lgb4', df_train, skf, hparams, config, lgb_adapter, rerun = 0)
np.mean(result['valid_scores']), result['valid_scores']

Fold:   0%|          | 0/5 [00:00<?, ?it/s]

/home/sun9sun9/python312/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/home/sun9sun9/python312/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/home/sun9sun9/python312/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/home/sun9sun9/python312/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/home/sun9sun9/python312/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  

(0.05995094205758729,
 [0.060346750917520384,
  0.05996893565243282,
  0.06038404435264214,
  0.05963170912729574,
  0.05942327023804537])

# 주요 변수를 범주화합니다.

In [5]:
df_train['Age_c'] = (df_train['Age'] // 10).astype('str').astype('category')
df_train['Duration_c'] = df_train['Duration'].astype('str').astype('category')
df_train['Heart_Rate_c'] = df_train['Heart_Rate'].round(0).astype('str').astype('category')

# CB7

In [84]:
hparams = {
    'model_params': {'max_depth': 8, 'n_estimators': 1500, 'learning_rate': 0.07, 'colsample_bylevel': 0.5}, 
    'X_num': ['Age', 'Height', 'Weight', 'Duration_log', 'Heart_Rate', 'Body_Temp'], 
    'X_cat': ['Duration_c', 'Age_c', 'Heart_Rate_c', 'Sex'],
    'validation_fraction': 'oof'
}

result_cb7 = sc.cv_result('cb7', df_train, skf, hparams, config, cb_adapter, use_gpu = 0, rerun = 0)
np.mean(result_cb7['valid_scores']), result_cb7['valid_scores']

(0.05916590111123312,
 [0.059558781443810487,
  0.059212310926459906,
  0.059516423551886334,
  0.05884613648923064,
  0.058695853144778244])

In [48]:
result['model_result'][3]['valid_result'].idxmin()

metric  set         
RMSE    learn           1499
        validation_0    1499
        validation_1    1472
dtype: int64

In [55]:
((df_train['Weight'] / 5).round() * 5).nunique()

20

In [56]:
((df_train['Height'] / 5).round() * 5).nunique()

20

# CB8

In [6]:
df_train['Weight_c'] = ((df_train['Weight'] / 5).round() * 5).astype('str').astype('category')
df_train['Height_c'] = ((df_train['Height'] / 5).round() * 5).astype('str').astype('category')
df_train['Body_Temp_c'] = ((df_train['Body_Temp'] / 2).round(1) * 2).astype('str').astype('category')

In [7]:
hparams = {
    'model_params': {'max_depth': 8, 'n_estimators': 1800, 'learning_rate': 0.07, 'colsample_bylevel': 0.5}, 
    'X_num': ['Age', 'Height', 'Weight', 'Duration_log', 'Heart_Rate', 'Body_Temp'], 
    'X_cat': ['Age_c', 'Weight_c', 'Height_c', 'Sex'],
    'validation_fraction': 'oof'
}

result_cb8 = sc.cv_result('cb8', df_train, skf, hparams, config, cb_adapter, use_gpu = 0, rerun = 0)
np.mean(result_cb8['valid_scores']), result_cb8['valid_scores']

(0.059215809366050476,
 [0.05953568165243805,
  0.05916992512473774,
  0.05963953317204185,
  0.05890039282701297,
  0.05883351405402178])

In [88]:
root_mean_squared_error(
    df_train[target], (sc.read_prd('cb8') + sc.read_prd('cb7')) / 2
)

0.05909968291552166

# Ensemble

In [8]:
models = ['lgb2', 'lgb1', 'xgb1', 'xgb3', 'cb1', 'cb2', 'cb3', 'cb4', 'cb5', 'nn1', 'nn2', 'cb7', 'cb8']
X_model = [i + '_skf5' if i not in ['cb7', 'cb8'] else i for i in models]
df_stk = sc.read_prds(X_model, index = df_train.index).assign(
    Calories_Log = df_train[target],
    duration_bin = pd.qcut(df_train['Duration'], q = 10, labels = np.arange(0, 10))
)

In [9]:
X_stk = [i + '_skf5' if i not in ['cb7', 'cb8'] else i for i in models]
hparams = {
    'X_num': X_stk
}
result = sgml.cv(df_stk, skf, hparams,  {**config, 'sp_y': 'duration_bin'}, lr_adapter)
np.mean(result['valid_scores'])

Fold:   0%|          | 0/5 [00:00<?, ?it/s]

0.058976317507087664

In [9]:
import sgfs
X_sel, _, result = sgfs.step_fs_fast(df_stk.assign(const = 1), X_stk, target, [], set(), [np.inf], root_mean_squared_error)
result

[inf,
 0.05945302738509161,
 0.05920054755556721,
 0.05911803675291648,
 0.05910277345841237,
 0.0590939597991154,
 0.05909040268995622,
 0.05908477292433439,
 0.059084709912654264]

In [10]:
result = sgml.cv(df_stk, skf, {'X_num': X_sel}, {**config, 'sp_y': 'duration_bin'}, lr_adapter, result_proc = [sgml.lr_learning_result])
np.mean(result['valid_scores']), result['valid_scores']

Fold:   0%|          | 0/5 [00:00<?, ?it/s]

(0.05898731902269151,
 [0.05933213662324541,
  0.05899660190394944,
  0.05939152257119935,
  0.05870712765910377,
  0.05850920635595958])

In [11]:
pd.concat([pd.Series(i['coef']) for i in result['model_result']], axis = 1).mean(axis = 1)

lgb2_skf5   -0.047926
cb5_skf5     0.070504
xgb3_skf5    0.247522
cb2_skf5     0.147886
lgb1_skf5    0.152489
xgb1_skf5   -0.021158
nn2_skf5     0.319530
cb1_skf5     0.131216
dtype: float64

In [10]:
for i in models:
    sc.train_cv(i, df_train, config)

I0000 00:00:1748187483.037219  259059 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5520 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4070 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


In [11]:
from sklearn.linear_model import LinearRegression
reg_meta_lr = LinearRegression()
reg_meta_lr.fit(df_stk[X_stk], df_train[target])

LinearRegression()

In [15]:
df_test['Age_c'] = (df_test['Age'] // 10).astype('str').astype('category')
df_test['Duration_c'] = df_test['Duration'].astype('str').astype('category')
df_test['Heart_Rate_c'] = df_test['Heart_Rate'].round(0).astype('str').astype('category')
df_test['Weight_c'] = ((df_test['Weight'] / 5).round() * 5).astype('str').astype('category')
df_test['Height_c'] = ((df_test['Height'] / 5).round() * 5).astype('str').astype('category')
df_test['Body_Temp_c'] = ((df_test['Body_Temp'] / 2).round(1) * 2).astype('str').astype('category')

In [16]:
df_stk_test = pd.concat([
    sc.get_predictor_cv(i, config)(df_test).rename(i) for i in X_stk
], axis = 1)
df_stk_test.head()

,lgb2_skf5,lgb1_skf5,xgb1_skf5,xgb3_skf5,cb1_skf5,cb2_skf5,cb3_skf5,cb4_skf5,cb5_skf5,nn1_skf5,nn2_skf5,cb7,cb8
id,,,,,,,,,,,,,
750000,3.342220,3.343804,3.336766,3.334744,3.339252,3.345108,3.339329,3.339992,3.335049,3.348824,3.343099,3.343146,3.339361
750001,4.694324,4.696249,4.704313,4.698164,4.691551,4.690045,4.690970,4.688648,4.685364,4.690432,4.693828,4.692813,4.689370
750002,4.471508,4.471234,4.473776,4.470401,4.471377,4.470860,4.481466,4.474276,4.484206,4.480945,4.477394,4.481249,4.479620
750003,4.837693,4.838142,4.835355,4.844798,4.840599,4.844425,4.841232,4.840612,4.838311,4.840408,4.839946,4.838162,4.843956
750004,4.335305,4.346876,4.339695,4.341092,4.345177,4.347627,4.345495,4.343181,4.333550,4.346485,4.347324,4.346172,4.349480


In [17]:
pd.Series(
    reg_meta_lr.predict(df_stk_test), index = df_stk_test.index, name = 'Calories'
).pipe(lambda x: (np.exp(x) - 1).clip(lower = 1)).to_csv('result/submission11.csv')

In [18]:
!head result/submission11.csv

id,Calories
750000,27.242263879150215
750001,108.23459316095095
750002,86.93627135613389
750003,125.63910649507822
750004,76.20354531438957
750005,21.74633320373594
750006,48.724038268298976
750007,6.774920100593104
750008,10.01742114034435


In [19]:
# !kaggle competitions submit -c playground-series-s5e5 -f result/submission11.csv -m "Ensemble11"

100%|██████████████████████████████████████| 6.06M/6.06M [00:02<00:00, 2.39MB/s]
Successfully submitted to Predict Calorie Expenditure